In [2]:
# imports
import pandas as pd
import numpy as np
import requests
import os
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
# Definining API key credentials
API_KEY =os.environ.get("FOURSQUARE_API_KEY")

In [5]:
# importing result from citybikes API with all stations in Toronto
df = pd.read_csv('/Applications/Lighthouse/Final-Project-Statistical-Modelling-with-Python/data/Bike_stations.csv', delimiter=',')

In [11]:
# converting Dataframe into json
Bike_stations = df.to_json(orient='records')

In [7]:
 # Defining API endpoint
url = "https://api.foursquare.com/v3/places/search"

# headers
headers = {"Accept": "application/json",
           "Authorization": API_KEY}



In [12]:
# Bike_stations was in str datatype after converting it to json
# to iterate through it change it to list of dictionary
Bike_stations = json.loads(Bike_stations)
#type(Bike_stations)


In [13]:
# creating empty list to store responses
response_list = []

# iteraing through each station in Bike_stations 
for station in Bike_stations:
    latitude = station.get('Latitude')
    longitude = station.get('Longitude')
    
    
     # Defining API endpoint
    url = "https://api.foursquare.com/v3/places/search"

    # headers
    headers = {"Accept": "application/json",
                 "Authorization": API_KEY}
    

    
    
    # setting up parameters for API request

    params = {
           'query': 'Indian Restaurant',
           'll': f"{latitude},{longitude}",
           'radius': 1000,
           'open_now': True,
           'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'
             }  
    
    # send the foresquare API request
    response = requests.get(url,params=params, headers=headers)
    
    
    
    # check if request was successfull (status code 200)
    if response.status_code == 200:
        #use the .json() method to parse the response content into json
        data_json = response.json()
        
        # Append the response to the list
        response_list.append(data_json)
    else:
        print(f"Request failed: {response.status_code}")

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [15]:
flattened_data = [
    {
    "Latitude": response["geocodes"]["main"].get("latitude"),
    "Longitude": response["geocodes"]["main"].get("longitude"),
    "name": response.get("name"),
    "distance": response.get("distance"),
    "address": response["location"].get("formatted_address"),
    "telephone": response.get("tel"),
    "rating": response.get("rating"),
    "open_true": response["hours"].get("open_now")
    }
  for result in response_list
  for response in result.get("results", [])
]

Put your parsed results into a DataFrame

In [16]:
fsq_restaurant = pd.DataFrame(flattened_data)

In [18]:
fsq_restaurant.shape

(3523, 8)

In [21]:
# creating a csv file for Bike_stations Dataframe to use it for further steps
file_path = '/Applications/Lighthouse/Final-Project-Statistical-Modelling-with-Python/data/fsq_restaurant.csv'

# save the dataframe into CSV file path
fsq_restaurant.to_csv(file_path, index=False)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [22]:

YELP_API_KEY = os.getenv("YELP_API_KEY")

#https://api.yelp.com/v3/businesses/search

In [24]:
# creating empty list to store responses
yelp_response_list = []

# maximum number of request to send
MAX_REQUEST = 490

# initialize a request count
request_count = 0


# iteraing through each station in Bike_stations 
for station in Bike_stations:
    
    #check if the maximum number of request has been reached
    if request_count >= MAX_REQUEST:
        break
        
    latitude = station.get('Latitude')
    longitude = station.get('Longitude')
    
    
     # Defining API endpoint
    url_yelp = "https://api.yelp.com/v3/businesses/search"

    # headers
    headers = {"Accept": "application/json",
                 "Authorization": f'Bearer {YELP_API_KEY}',
                 }
    

    
    
    # setting up parameters for API request

    params = {
           'latitude': latitude,
           'longitude': longitude,
           'term': 'Restaurant',
           'radius': 1000,
             }  
    
    # send the foresquare API request
    response_yelp = requests.get(url=url_yelp,params=params, headers=headers)
    
    
    # check if request was successfull (status code 200)
    if response_yelp.status_code == 200:
        #use the .json() method to parse the response content into json
        yelp_data_json = response_yelp.json()
        
        # Append the response to the list
        yelp_response_list.append(yelp_data_json)
        
        # Increment the request counter
        request_count += 1
    else:
        print(f"Request failed: {response_yelp.status_code}")

Request failed: 500


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [37]:
#yelp_response_list.get['businesses'][0]['coordinates']
coordinates_list=[]
for record in yelp_response_list:
    for business in record.get('businesses', []):
        coordinates = business.get('coordinates', {})
        if 'latitude' in coordinates and 'longitude' in coordinates:
            latitude = coordinates['latitude']
            longitude = coordinates['longitude']
            coordinates_list.append((latitude, longitude))

# Print all coordinates
#for latitude, longitude in coordinates_list:
   # print(f'Latitude: {latitude}, Longitude: {longitude}')
#This code assumes that 'yelp_response_list' is a list of dictionaries and handles cases where 'businesses' or 'coordinates' may be missing or have unexpected structures.

In [40]:
yelp_flattened_data = [
    {
    "name": response.get("name"),
    "distance": response.get("distance"),
    "latitude": response["coordinates"].get("latitude"),
    "longitude": response["coordinates"].get("longitude"),
    "address": response["location"].get("display_address"),
    "telephone": response.get("display_phone"),
    "rating": response.get("rating"),
    "review_counts": response.get("review_count"),
    "closed": response.get("is_closed")
    }
  for result in yelp_response_list
  for response in result.get("businesses", [])
]

Put your parsed results into a DataFrame

In [41]:
yelp_restaurant = pd.DataFrame(yelp_flattened_data)

In [43]:
# creating a csv file for Bike_stations Dataframe to use it for further steps
file_path = '/Applications\Lighthouse\/Final-Project-Statistical-Modelling-with-Python\data\yelp_restaurant.csv'

# save the dataframe into CSV file path
yelp_restaurant.to_csv(file_path, index=False)

In [44]:
yelp_restaurant = pd.read_csv('/Applications\Lighthouse\/Final-Project-Statistical-Modelling-with-Python\data\yelp_restaurant.csv')

In [46]:
yelp_restaurant.shape

(9531, 9)

# Comparing Results

We will go over indian_restaurant result that is the result we got from Foursqare API

In [52]:
fsq_restaurant

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
0,43.672665,-79.321041,Udupi Palace,810,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",(416) 405-8189,8.6,True
1,43.672278,-79.322523,Motimahal Restaurant Ltd,798,"1422 Gerrard St E (Coxwell), Toronto ON M4L 1Z6",(416) 461-3111,7.6,True
2,43.671871,-79.324283,Bombay Chowpatty,812,"1386 Gerrard St E, Toronto ON M4L 1Z2",(416) 405-8080,7.3,True
3,43.672004,-79.322433,Regency Restaurant Inc,790,"1423 Gerrard St E (Hiawatha St), Toronto ON M4...",(416) 778-7366,6.2,True
4,43.666400,-79.316826,Butter Chicken Roti,255,"1610 Queen St E (Coxwel), Toronto ON M4L 1G2",(647) 349-7684,NaN,True
...,...,...,...,...,...,...,...,...
3518,43.641873,-79.411437,My Roti Place,982,"901 King St W (at Strachan Ave), Toronto ON M5...",(647) 295-9446,NaN,True
3519,43.638479,-79.417204,Brazen Head Irish Pub,617,"165 E. Liberty St (at Lynn Williams St.), Toro...",(416) 535-8787,6.1,True
3520,43.641873,-79.411437,My Roti Place,622,"901 King St W (at Strachan Ave), Toronto ON M5...",(647) 295-9446,NaN,True
3521,43.638768,-79.416315,Maurya East Indian Cuisine - Liberty Village,655,"150 E Liberty St, Toronto ON M6K 3R5",(647) 347-7002,NaN,True


In [53]:
# check any null values
fsq_restaurant.isnull().sum()

Latitude        0
Longitude       0
name            0
distance        0
address         0
telephone       4
rating       1423
open_true       0
dtype: int64

In [54]:
# checking duplicating values 
fsq_restaurant.duplicated().sum()

65

In [55]:
fsq_restaurant.shape

(3523, 8)

We will go over indian restaurants that we got from yelp API requests

In [56]:
yelp_restaurant.shape

(9531, 9)

In [57]:
# checking null values
yelp_restaurant.isnull().sum()

name               0
distance           0
latitude           0
longitude          0
address            0
telephone        735
rating             0
review_counts      0
closed             0
dtype: int64

In [58]:
# checking duplicate values
yelp_restaurant.duplicated().sum()

0

Which API provided you with more complete data? Provide an explanation. 

Yelp API provided more complete data as mentioned in above steps, There are no null and duplicated value in yelp result, where in foursquare there are null results in ratings and telephone numbers, also yelp provided more field atrributes from results then foursquare and yelp returns more precise and quantity result then foursqare.

One downside for yelp is that it provides less api requests then Foursqare as you have to me more carefull with code writing with yelp request as to not run out of requests

Get the top 10 restaurants according to their rating

In [61]:
# indian_restaurants foursquare result 
fsq_restaurant.sort_values(by='rating',ascending=False).head(10)

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
1725,43.681122,-79.429253,Pukka,779,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
812,43.681122,-79.429253,Pukka,869,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
502,43.681122,-79.429253,Pukka,363,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
2680,43.681122,-79.429253,Pukka,590,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
1630,43.681122,-79.429253,Pukka,695,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
2688,43.681122,-79.429253,Pukka,452,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
1523,43.681122,-79.429253,Pukka,124,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
1921,43.681122,-79.429253,Pukka,854,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
2321,43.681122,-79.429253,Pukka,913,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True
1681,43.681122,-79.429253,Pukka,272,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.2,True


In [60]:
# indian restaurant in yelp API
yelp_restaurant.sort_values(by='rating', ascending=False).head(10)

,name,distance,latitude,longitude,address,telephone,rating,review_counts,closed
2215,Ladybug Wine Bar,531.176279,43.656593,-79.357109,"['514 King Street E', 'Toronto, ON M5A 1M1', '...",NaN,5.0,3,False
7170,Old Town Bodega,846.750219,43.654200,-79.360990,"['402 King St E', 'Toronto, ON M5A 1L4', 'Cana...",NaN,5.0,9,False
7233,Slayer Burger,59.909211,43.701051,-79.396936,"['2013 Yonge Street', 'Toronto, ON M4S 1Z8', '...",+1 416-846-3786,5.0,7,False
7231,Samba Brazil Eatery,300.438829,43.703299,-79.397444,"['2109 Yonge Street', 'Toronto, ON M4S 2A4', ...",+1 437-880-7808,5.0,1,False
4187,Rikki Tikki,627.646346,43.654081,-79.401486,"['209 Augusta Avenue', 'Toronto, ON M5T 2L4', ...",+1 416-792-3229,5.0,27,False
7214,Gurume Sushi,755.419915,43.661391,-79.380995,"['45 Carlton Street', 'unit 5', 'Toronto, ON M...",+1 416-901-8333,5.0,56,False
7201,Momo Ghar,408.060026,43.667410,-79.369530,"['568 Parliament St', 'Toronto, ON M4X 1P6', '...",+1 416-920-8224,5.0,13,False
7198,Afghano Restaurant,668.858582,43.655715,-79.364637,"['344 Queen Street E', 'Unit 1', 'Toronto, ON ...",+1 647-906-2269,5.0,1,False
4213,Haidilao Hot Pot,324.541353,43.654633,-79.379839,"['237 Yonge Street', 'Toronto, ON M5B 1N8', 'C...",+1 437-778-1068,5.0,34,False
4215,Gurume Sushi,441.165056,43.661391,-79.380995,"['45 Carlton Street', 'unit 5', 'Toronto, ON M...",+1 416-901-8333,5.0,56,False
